## Stream attributes from Web events using Signals

This notebook creates a new view using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [6]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()
# You will need valid BDP credentials (API key, API key ID and org ID) for this to work.
# Instructions on how to generate these here: https://docs.snowplow.io/docs/account-management/managing-console-api-authentication/#credentials-ui-v3
sp_signals = Signals(
    api_url="http://localhost:8008", # Local Signals API endpoint
    api_key=os.environ["SNOWPLOW_API_KEY"],
    api_key_id=os.environ["SNOWPLOW_API_KEY_ID"],
    org_id=os.environ["SNOWPLOW_ORG_ID"],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [215]:
from snowplow_signals import (
    Attribute,
    Criteria,
    Criterion,
    Event,
)

variant_click_count = Attribute(
    name="variant_click_count",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="button_click",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    property="unstruct_event_com_snowplowanalytics_snowplow_button_click_1:label",
    criteria=Criteria(
        all=[
            Criterion(
                property="event_name",
                operator="=",
                value="button_click",
            ),
        ],
    ),
)

# a dummy event is defined for this attribute as we only use this for storing an intervention value so we don't want signals to write a value to it
intervention_feature = Attribute(
    name="intervention_example",
    type="string",
    events=[
        Event(
            vendor="doesnotexist.com",
            name="anything",
            version="1-0-0"
        )
    ],
    criteria=Criteria(
        all=[
            Criterion(
                property="page_urlpath",
                operator="=",
                value="foobar",
            ),
        ]
    ),
    aggregation="first"
)

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes.

Views are immutable and versioned.

In [ ]:
from snowplow_signals import View, session_entity, Service

view = View(
    name="demo_attributes",
    version=1,
    entity=session_entity,
    attributes=[
        variant_click_count, # a count of link click events
        intervention_feature # stores the result of our intervention once triggered
    ],
    online=True
)

service = Service(
    name="demo_service",
    description='Service for demoing signals and inteventions',
    views=[view]
)

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [217]:
# Ensure Snowplow Local and Personalization API are running otherwise this will fail
applied = sp_signals.apply([view, service])
print(f"{len(applied)} objects applied")
# If successful you should see two objects applied (one for the view and one for the service).

2 objects applied


In [ ]:
# check the values

sid = '' # enter your domain_sessionid here from the Shopify store (https://quickstart-b5d55a69.myshopify.com/) to check the value
# of your features ahead of time
import requests
r = requests.post(
    'http://localhost:6566/get-online-features', # Personalization API must be up. Auth not required!
    json = {
        'entities': {
            'session': [sid]
        },
        'feature_service': 'demo_service'
    }
)
# print out a prettier version
features = {}
feature_names = r.json()['metadata']['feature_names']
feature_values = r.json()['results']
for i, feature in enumerate(feature_names):
    features[feature] = feature_values[i]['values']
print(features)

{'domain_sessionid': ['78348adf-ff98-4448-b8d4-8d20c4ae42ab'], 'variant_click_count': [7], 'intervention_example': ['what is up pussycat']}


In [ ]:
# now let's create an intervention using the API
# this intervention will set an attribute for `intervention_example` if the intervention is triggered

number_clicks = 2 # trigger the intervention when variants clicks is this value or greater

intervention = {
    'name': 'unsure_clicker3',
    'version': 1,
    'method': 'set_attribute',
    'context': {
        'attribute': 'demo_attributes:intervention_example',
        'value': 'demo message',
        'clear_history': True
    },
    'criteria': {'operator': '>=', 'value': number_clicks, 'attribute': 'demo_attributes:variant_click_count'},
}

r = requests.post(
    'http://localhost:8008/api/v1/registry/interventions/',
    json=intervention,
    headers={
        'Content-Type': 'application/json',
        'Authorization': 'Bearer NA' # Auth not required
    }
)
print(r.status_code) # ensure the code printed out is 200

200


Go to the Shopify store for this product ( https://quickstart-b5d55a69.myshopify.com/ ) and click around the sizes until you see at least [number_clicks] button_click events fire in the Chrome Inspector.

Wait 5-10 seconds and you should see the intervention pop up on the site.